# Cancer Classification
This notebook trains classifiers to detect cancer in MRI scans, using automated labels from the LLM pipeline.

In [1]:
import os
import pickle
import misvm
import torch
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, balanced_accuracy_score, roc_curve
from imblearn.over_sampling import RandomOverSampler

ivd_arrays_path = '/work/robinpark/NCIMI_clean/ncimi_ivd_arrays/april2024_splits'

# SEED
seed=0
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [2]:
# Pickle the dictionary
with open(f'{ivd_arrays_path}/ncimi_samples_dict.pkl', 'rb') as handle:
    samples = pickle.load(handle)

In [3]:
train_samples = samples['train_samples']
val_samples = samples['val_samples']
test_samples = samples['test_samples']

In [4]:
# Summarise splits
def sum_samples(samples):
    df = pd.DataFrame.from_dict(samples, orient='index', columns=['results']).reset_index()

    # Split index into columns
    df[['pat_id','stu_id','ser_id','level1','level2']] = df['index'].str.split('_',expand=True)

    # Sum rows by pat_id, date, level, and unique results 
    df = df.groupby(['pat_id','stu_id','ser_id','level1','level2', 'results']).size().reset_index(name='counts')

    df['pat_stu_id'] = df['pat_id'] + '_' + df['stu_id']

    print('unique pat:', len(df[['pat_id']].drop_duplicates()))
    print('unique studies:', len(df[['pat_stu_id']].drop_duplicates()))

    display(df.groupby(['results'])[['counts']].sum())

In [5]:
sum_samples(train_samples)

unique pat: 1235
unique studies: 1265


,counts
results,
0,5337
1,8924


In [6]:
sum_samples(val_samples)

unique pat: 312
unique studies: 318


,counts
results,
0,1281
1,2237


In [7]:
sum_samples(test_samples)

unique pat: 450
unique studies: 451


,counts
results,
0,1798
1,3120


## ResNet18 Encodings + SVC

In [8]:
with open(f'{ivd_arrays_path}/ncimi_resnet_encodings.pkl', 'rb') as handle:
    ncimi_resnet_encodings = pickle.load(handle)

train_features_cpu = ncimi_resnet_encodings['train_features_cpu']
label_train_array = ncimi_resnet_encodings['label_train_array']

val_features_cpu = ncimi_resnet_encodings['val_features_cpu']
label_val_array = ncimi_resnet_encodings['label_val_array']

test_features_cpu = ncimi_resnet_encodings['test_features_cpu']
label_test_array = ncimi_resnet_encodings['label_test_array']

In [9]:
# Load pickled arrays
with open(f'{ivd_arrays_path}/ncimi_arrays_dict.pkl', 'rb') as handle:
    ncimi_array_dict = pickle.load(handle)

label_test_report = ncimi_array_dict['label_test_report']
label_test_scores = ncimi_array_dict['label_test_scores']
label_test_con = ncimi_array_dict['label_test_con']
compare_test_array = ncimi_array_dict['compare_test_array']

test_pat_id_date = ncimi_array_dict['test_pat_id_date']
val_pat_id_date = ncimi_array_dict['val_pat_id_date']
train_pat_id_date = ncimi_array_dict['train_pat_id_date']

In [10]:
# Standardise
scaler = StandardScaler()
train_features_cpu = scaler.fit_transform(train_features_cpu)
val_features_cpu = scaler.transform(val_features_cpu)
test_features_cpu = scaler.transform(test_features_cpu)

In [11]:
train_features_cpu = [torch.Tensor(i) for i in train_features_cpu]
val_features_cpu = [torch.Tensor(i) for i in val_features_cpu]
test_features_cpu = [torch.Tensor(i) for i in test_features_cpu]

### SVC Using Max Bag

In [12]:
def get_max_bag(features, labels, patient_slices):
    # Combine features, labels, and patient slices into a DataFrame
    df = pd.DataFrame({'Features': features, 'Labels': labels, 'Patient_Slices': patient_slices})
    
    # Group by patient
    grouped = df.groupby('Patient_Slices')
    
    mil_bags = []
    mil_label = []
    
    for patient_slice, group in grouped:
        # Combine slices into a bag
        bag_features = group['Features'].tolist()
        
        bag_label = group['Labels'].max()
        # if bag_label == 0:
        #     bag_label = -1
        
        # Add bag to MIL bags
        mil_bags.append(bag_features)
        mil_label.append(bag_label)

    max_bags = []
    for bag in mil_bags:
        max_bag, _ = torch.max(torch.stack(bag), dim=0)
        max_bags.append(max_bag)
    
    return max_bags, mil_label

In [13]:
train_max_bags, train_labels = get_max_bag(train_features_cpu, label_train_array, train_pat_id_date)
test_max_bags, test_labels = get_max_bag(test_features_cpu, label_test_array, test_pat_id_date)
val_max_bags, val_labels = get_max_bag(val_features_cpu, label_val_array, val_pat_id_date)

#### Resample Samples

In [14]:
# Count the values in train_lables
unique, counts = np.unique(test_labels, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  0 172]
 [  1 279]]


In [15]:
# Separate positive and negative bags
pos_train_bags = [bag for bag, label in zip(train_max_bags, train_labels) if label == 1]
neg_train_bags = [bag for bag, label in zip(train_max_bags, train_labels) if label == 0]

In [16]:
# Double the negative examples to balance
neg_train_bags_rs = neg_train_bags + neg_train_bags[:round(len(neg_train_bags)*.65)]

train_max_bags_rs = pos_train_bags + neg_train_bags_rs
train_labels_rs = [1] * len(pos_train_bags) + [0] * len(neg_train_bags_rs)

In [17]:
len(neg_train_bags_rs)

810

In [18]:
# Count the values in train_lables
unique, counts = np.unique(train_labels_rs, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  0 810]
 [  1 774]]


In [19]:
# Initialize SVM classifier
svm_classifier = svm.SVC(kernel='linear', C=100, probability=True)

# Train the classifier
svm_classifier.fit(train_max_bags_rs, train_labels_rs)

SVC(C=100, kernel='linear', probability=True)

In [20]:
# Predict the validation set
val_pred = svm_classifier.predict(val_max_bags)
val_prob = svm_classifier.predict_proba(val_max_bags)[:, 1]

# Evaluate using AUROC, F1 and balanced accuracy
val_auc = roc_auc_score(val_labels, val_prob)
val_f1 = f1_score(val_labels, val_pred)
val_bal_acc = balanced_accuracy_score(val_labels, val_pred)
val_acc = np.mean(val_pred == val_labels)

fpr, tpr, thresholds = roc_curve(val_labels, val_prob)

# Find the point where FPR equals FRR
val_eer_threshold = thresholds[np.nanargmin(np.absolute((1 - tpr) - fpr))]
val_eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]  # Equal to FRR

print(f'Val Accuracy: {val_acc:.3f}')
print(f'Val AUC: {val_auc:.3f}')
print(f'Validation F1: {val_f1:.3f}')
print(f'Validation Balanced Accuracy: {val_bal_acc:.3f}')
print(f'EER: {val_eer:.3f}')
print(f'EER Threshold: {val_eer_threshold:.3f}\n')

# Predict test set
test_prob = svm_classifier.predict_proba(test_max_bags)[:, 1]

fpr, tpr, thresholds = roc_curve(test_labels, test_prob)

# Find the point where FPR equals FRR
test_eer_threshold = thresholds[np.nanargmin(np.absolute((1 - tpr) - fpr))]
test_eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]  # Equal to FRR

test_pred = test_prob > val_eer_threshold

# Evaluate using accuracy, F1 and balanced accuracy
test_auc = roc_auc_score(test_labels, test_prob)
test_f1 = f1_score(test_labels, test_pred)
test_bal_acc = balanced_accuracy_score(test_labels, test_pred)
test_acc = np.mean(test_pred == test_labels)

print(f'Test Accuracy: {test_acc:.3f}')
print(f'Test AUC: {test_auc:.3f}')
print(f'Test F1: {test_f1:.3f}')
print(f'Test Balanced Accuracy: {test_bal_acc:.3f}')
print(f'EER: {test_eer:.3f}')
print(f'EER Threshold: {test_eer_threshold:.3f}')

Val Accuracy: 0.648
Val AUC: 0.708
Validation F1: 0.702
Validation Balanced Accuracy: 0.640
EER: 0.369
EER Threshold: 0.582

Test Accuracy: 0.659
Test AUC: 0.711
Test F1: 0.705
Test Balanced Accuracy: 0.658
EER: 0.343
EER Threshold: 0.592


### MIL-SVM

In [21]:
def create_mil_bags(features, labels, patient_slices):
    # Combine features, labels, and patient slices into a DataFrame
    df = pd.DataFrame({'Features': features, 'Labels': labels, 'Patient_Slices': patient_slices})
    
    # Group by patient
    grouped = df.groupby('Patient_Slices')
    
    mil_bags = []
    mil_label = []
    
    for patient_slice, group in grouped:
        # Combine slices into a bag
        bag_features = group['Features'].tolist()
        
        bag_label = group['Labels'].max()
        if bag_label == 0:
            bag_label = -1
        
        # Add bag to MIL bags
        mil_bags.append(bag_features)
        mil_label.append(bag_label)
    
    return mil_bags, mil_label

In [22]:
train_bags, train_labels = create_mil_bags(train_features_cpu, label_train_array, train_pat_id_date)
test_bags, test_labels = create_mil_bags(test_features_cpu, label_test_array, test_pat_id_date)
val_bags, val_labels = create_mil_bags(val_features_cpu, label_val_array, val_pat_id_date)

In [23]:
def eval_mil(model):

    mil_val_scores = model.predict(val_bags)
    mil_val_pred = np.sign(mil_val_scores)

    fpr, tpr, thresholds = roc_curve(val_labels, mil_val_scores)

    # Find the point where FPR equals FRR
    val_eer_threshold = thresholds[np.nanargmin(np.absolute((1 - tpr) - fpr))]
    val_eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]  # Equal to FRR

    print('Validation Balanced Accuracy:', balanced_accuracy_score(val_labels, mil_val_pred))
    print('Validation F1 Score:', f1_score(val_labels, mil_val_pred))
    print('Validation AUC:', roc_auc_score(val_labels, mil_val_scores))
    print('Validation EER:', val_eer)
    print('Validation EER Threshold:', val_eer_threshold)

    mil_test_scores = model.predict(test_bags)
    mil_test_pred = np.where(mil_test_scores > val_eer_threshold, 1, -1)

    fpr, tpr, thresholds = roc_curve(test_labels, mil_test_scores)

    # Find the point where FPR equals FRR
    test_eer_threshold = thresholds[np.nanargmin(np.absolute((1 - tpr) - fpr))]
    test_eer = fpr[np.nanargmin(np.absolute((1 - tpr) - fpr))]  # Equal to FRR

    print('\nTest Balanced Accuracy:', balanced_accuracy_score(test_labels, mil_test_pred))
    print('Test F1 Score:', f1_score(test_labels, mil_test_pred))
    print('Test AUC:', roc_auc_score(test_labels, mil_test_scores))
    print('Test EER:', test_eer)
    print('Test EER Threshold:', test_eer_threshold)

    return mil_test_pred, mil_test_scores

### Resample

In [24]:
# Separate positive and negative bags
pos_train_bags = [bag for bag, label in zip(train_bags, train_labels) if label == 1]
neg_train_bags = [bag for bag, label in zip(train_bags, train_labels) if label == -1]

In [25]:
# Double the negative examples to balance
neg_train_bags_rs = neg_train_bags + neg_train_bags[:round(len(neg_train_bags)*.65)]

train_bags_rs = pos_train_bags + neg_train_bags_rs
train_labels_rs = [1] * len(pos_train_bags) + [-1] * len(neg_train_bags_rs)

In [26]:
# Count the values in train_lables
unique, counts = np.unique(train_labels_rs, return_counts=True)
print(np.asarray((unique, counts)).T)

[[ -1 810]
 [  1 774]]


### NSK-SVM
Maps entire bags to features, using distance between two bags to generate the kernel (Gärtner et al, 2002)

In [27]:
nsk_svm = misvm.NSK(kernel='linear_av', C=10)
nsk_svm.fit(train_bags_rs, train_labels_rs)

Setup QP...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -2.9795e+02 -3.5238e+01  1e+04  1e+02  1e-12
 1: -3.2813e+01 -2.7412e+01  7e+02  6e+00  1e-12
 2: -7.3862e+00 -2.2677e+01  7e+01  5e-01  1e-13
 3: -4.9144e+00 -1.5330e+01  1e+01  6e-02  2e-14
 4: -5.0589e+00 -7.0856e+00  2e+00  8e-03  9e-15
 5: -5.4928e+00 -6.0514e+00  6e-01  2e-03  1e-14
 6: -5.6294e+00 -5.8258e+00  2e-01  3e-04  9e-15
 7: -5.6904e+00 -5.7297e+00  4e-02  5e-05  1e-14
 8: -5.7045e+00 -5.7088e+00  4e-03  2e-06  1e-14
 9: -5.7062e+00 -5.7066e+00  4e-04  2e-07  1e-14
10: -5.7064e+00 -5.7064e+00  1e-05  6e-09  1e-14
11: -5.7064e+00 -5.7064e+00  2e-06  9e-10  1e-14
Optimal solution found.


In [28]:
mil_test_pred, mil_test_scores = eval_mil(nsk_svm)

Validation Balanced Accuracy: 0.757694881231181
Validation F1 Score: 0.7891891891891892
Validation AUC: 0.8404566744730678
Validation EER: 0.2459016393442623
Validation EER Threshold: -0.009485154626617337

Test Balanced Accuracy: 0.763367925314662
Test F1 Score: 0.7729083665338645
Test AUC: 0.8513378344586147
Test EER: 0.21511627906976744
Test EER Threshold: -0.21384291030091582


## Visualise Resnet18 Encodings + NSK-SVM Results

In [29]:
# Load pickled arrays
with open(f'{ivd_arrays_path}/ncimi_arrays_dict.pkl', 'rb') as handle:
    ncimi_array_dict = pickle.load(handle)

ivd_train_array = ncimi_array_dict['ivd_train_array']
label_train_array = ncimi_array_dict['label_train_array']

ivd_val_array = ncimi_array_dict['ivd_val_array']
label_val_array = ncimi_array_dict['label_val_array']

ivd_test_array = ncimi_array_dict['ivd_test_array']
label_test_array = ncimi_array_dict['label_test_array']
label_test_report = ncimi_array_dict['label_test_report']
label_test_scores = ncimi_array_dict['label_test_scores']
label_test_con = ncimi_array_dict['label_test_con']
compare_test_array = ncimi_array_dict['compare_test_array']
ivd_test_array_names = ncimi_array_dict['ivd_test_array_names']

In [30]:
def mil_level_report(
    label_test_report, label_test_con, label_test_scores, 
    compare_test_array, ivd_test_array, ivd_test_array_names, 
    patient_slices):
    # Combine features, labels, and patient slices into a DataFrame
    df = pd.DataFrame(
        {'Reports': label_test_report, 
        'Conclusions': label_test_con, 
        'Label Scores': label_test_scores,
        'Predicted Label': compare_test_array,
        'IVD Test Array': ivd_test_array,
        'IVD Names': ivd_test_array_names,
        'Patient Slices': patient_slices})

    # Group by patient
    grouped = df.groupby('Patient Slices')
    
    bag_report = []
    bag_con = []
    bag_score = []
    bag_pred_label = []
    bag_ivd_test_array = []
    bag_names = []
    
    for patient_slice, group in grouped:
        # Combine slices into a bag
        
        bag_report.append(group['Reports'].iloc[0])
        bag_con.append(group['Conclusions'].iloc[0])
        bag_score.append(group['Label Scores'].iloc[0])
        bag_pred_label.append(group['Predicted Label'].iloc[0])
        bag_ivd_test_array.append(group['IVD Test Array'].tolist())
        bag_names.append(group['IVD Names'].tolist())
    
    return bag_report, bag_con, bag_score, bag_pred_label, bag_ivd_test_array, bag_names

(df_lab_test_report, 
 df_lab_test_con, 
 df_lab_test_scores, 
 df_lab_pred_labels, 
 bag_ivd_test_array, 
 bag_of_names) = mil_level_report(label_test_report, 
                                  label_test_con, 
                                  label_test_scores, 
                                  compare_test_array, 
                                  ivd_test_array, 
                                  ivd_test_array_names, 
                                  test_pat_id_date)

In [31]:
# # Visualise outputs at each step 
# counter = 3
# zipped_results = list(zip(df_lab_test_report, df_lab_test_con, df_lab_test_scores, df_lab_pred_labels, test_labels, mil_test_pred, mil_test_scores, bag_ivd_test_array, bag_of_names))

# for report, pred_con, pred_report_score, pred_report_label, true_report_label, pred_scan_label, pred_scan_score, ivd, ivd_name in zipped_results:
#     counter += 1
#     if counter <7:
#         if pred_report_label == 1 and true_report_label == 1 and pred_scan_label == 1:
#             print(f'Report: {report}')
#             print(f'Conclusion: {pred_con}')
#             print(f'Predicted Report Score: {pred_report_score}')
#             print(f'Predicted Report Label: {pred_report_label}')
#             print(f'True Report Label: {true_report_label}')
#             print(f'Predicted Scan Label: {pred_scan_label}')
#             print(f'Predicted Scan Score: {pred_scan_score}')
#             # Plot ivd
#             for i in range(len(ivd)):
#                 print(f'IVD: {ivd_name[i]}')
#                 plt.imshow(ivd[i][3,:,:], cmap='gray')
#                 plt.show()
#             print('\n')
#     elif counter == 7:
#         break